In [110]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/vivino.db")
cursor = conn.cursor()

### 1. highlight 10 wines to increase our sales. Which ones should we choose and why?

In [111]:
query_1 = f"""
    SELECT wines.name, vintages.price_euros, vintages.ratings_average, rank
    FROM vintages
    JOIN vintage_toplists_rankings
    ON vintages.id = vintage_toplists_rankings.vintage_id
    JOIN wines
    ON vintages.wine_id = wines.id
    WHERE vintage_toplists_rankings.rank > vintage_toplists_rankings.previous_rank
    AND vintages.price_euros < (SELECT AVG(price_euros) FROM vintages) AND vintages.ratings_average > 4.0 AND rank > 10
    ORDER BY vintages.price_euros ASC, vintages.ratings_average DESC
    LIMIT 10;"""

df = pd.read_sql_query(query_1, conn)

df.index += 1

df

,name,price_euros,ratings_average,rank
1,60 Sessantanni Old Vines Primitivo di Manduria,24.75,4.4,17
2,60 Sessantanni Old Vines Primitivo di Manduria,24.75,4.4,19
3,Limited Edition 10 Vendemmie,34.20,4.5,15
4,Malleolus,35.95,4.4,18
5,Lupi Rezerva,36.55,4.6,15
6,Bramare Malbec Uco Valley,60.20,4.5,12
7,Tinto,64.95,4.4,20
8,Chardonnay,78.95,4.4,18
9,Malbec Argentino,89.95,4.5,15
10,Les Noisetiers,94.00,4.4,19


### 2. We have a limited marketing budget for this year. Which country should we prioritise and why?

##### `New column Added to countries for this question! (See script database_edits)`

In [112]:
query_2 = f"""
    SELECT name, users_count, wineries_count, proportion_consumer_wineries
    FROM countries
    WHERE users_count > 1000000
    ORDER BY proportion_consumer_wineries DESC
    LIMIT 3;"""
 


# cursor.execute(query_2)
# cursor.fetchall()

df = pd.read_sql_query(query_2, conn)

df.index += 1

df['users_count'] = df['users_count'].apply(lambda x: "{:,}".format(x))
df['wineries_count'] = df['wineries_count'].apply(lambda x: "{:,}".format(x))
df['proportion_consumer_wineries'] = df['proportion_consumer_wineries'].apply(lambda x: f"{x}%")

df


,name,users_count,wineries_count,proportion_consumer_wineries
1,États-Unis,"12,273,684","28,145",436.0%
2,Suisse,"1,601,799","3,849",416.0%
3,Portugal,"1,123,535","5,834",192.0%


### 3. We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

### 4. We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

### 5. We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.


In [113]:
# Find the three most used grapes
query_5 = f"""SELECT name, COUNT(name) AS occurrence_count
FROM (
    SELECT country_code, name
    FROM most_used_grapes_per_country
    JOIN grapes ON grapes.id = most_used_grapes_per_country.grape_id
) AS subquery
GROUP BY name
ORDER BY occurrence_count DESC
LIMIT 3;"""

df = pd.read_sql_query(query_5, conn)
df.index += 1

grape_names = df["name"].unique()
grape_names_str = ", ".join([f"'{name}'" for name in grape_names])

df


,name,occurrence_count
1,Cabernet Sauvignon,12
2,Merlot,11
3,Chardonnay,6


In [118]:
# Search the best 5 wines for Cabernet Sauvignon

query_5_1 = f"""
    SELECT wines.name AS vino_name, ratings_average, grapes.name AS grape_name
    FROM regions
    JOIN wines
    ON regions.id = wines.region_id
    JOIN most_used_grapes_per_country
    ON regions.country_code = most_used_grapes_per_country.country_code
    JOIN grapes
    ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE grape_name = "Cabernet Sauvignon" AND ratings_average > 4
    ORDER BY grape_name, ratings_average DESC
    LIMIT 5;"""


df1 = pd.read_sql_query(query_5_1, conn)
df1.index += 1
df1

,vino_name,ratings_average,grape_name
1,Cabernet Sauvignon,4.8,Cabernet Sauvignon
2,Mágico,4.8,Cabernet Sauvignon
3,IX Estate Red,4.7,Cabernet Sauvignon
4,Special Selection Cabernet Sauvignon,4.7,Cabernet Sauvignon
5,Unico Reserva Especial Edición,4.7,Cabernet Sauvignon


In [119]:
# Search the best 5 wines for Merlot

query_5_2 = f"""
    SELECT wines.name AS vino_name, ratings_average, grapes.name AS grape_name
    FROM regions
    JOIN wines
    ON regions.id = wines.region_id
    JOIN most_used_grapes_per_country
    ON regions.country_code = most_used_grapes_per_country.country_code
    JOIN grapes
    ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE grape_name = "Merlot" AND ratings_average > 4
    ORDER BY grape_name, ratings_average DESC
    LIMIT 5;"""


df2 = pd.read_sql_query(query_5_2, conn)
df2.index += 1
df2

,vino_name,ratings_average,grape_name
1,Amarone della Valpolicella Classico Riserva,4.8,Merlot
2,Fratini Bolgheri Superiore,4.8,Merlot
3,Cristal Rosé Vinothèque,4.8,Merlot
4,Batard-Montrachet Grand Cru,4.8,Merlot
5,Eszencia,4.7,Merlot


In [122]:
# Search the best 5 wines for Chardonnay

query_5_3 = f"""
    SELECT wines.name AS vino_name, ratings_average, grapes.name AS grape_name
    FROM regions
    JOIN wines
    ON regions.id = wines.region_id
    JOIN most_used_grapes_per_country
    ON regions.country_code = most_used_grapes_per_country.country_code
    JOIN grapes
    ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE grape_name = "Chardonnay" AND ratings_average > 4
    ORDER BY grape_name, ratings_average DESC
    LIMIT 5;"""


df3 = pd.read_sql_query(query_5_3, conn)
df3.index += 1
df3

,vino_name,ratings_average,grape_name
1,Amarone della Valpolicella Classico Riserva,4.8,Chardonnay
2,Cabernet Sauvignon,4.8,Chardonnay
3,Fratini Bolgheri Superiore,4.8,Chardonnay
4,Cristal Rosé Vinothèque,4.8,Chardonnay
5,Batard-Montrachet Grand Cru,4.8,Chardonnay


### 6. We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

### 7. One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?